In [1]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import numpy as np
import re

In [235]:

headers = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36'
}

# Step 1: Scrape listing links
i = 4245
url = f'https://www.otodom.pl/pl/wyniki/sprzedaz/mieszkanie/cala-polska?page={i}'
houselinks = []
while True:  # Adjust the range as needed
    i += 1
    r = requests.get(url, headers=headers)

    url = f'https://www.otodom.pl/pl/wyniki/sprzedaz/mieszkanie/cala-polska?page={i}'
    print(f'Page {i} status code:', r.status_code)
    soup = BeautifulSoup(r.content, 'lxml')

    # Check if no listings found
    h3_element = soup.find('h3', class_='css-1nw1os0 e1ws6l2x3')
    if h3_element and h3_element.get_text(strip=True) == 'Nie znaleźliśmy żadnych ogłoszeń':
        print("No advertisements found. Stopping the scraping process.")
        break

    listing_container = soup.find_all('section', class_='eeungyz1 css-hqx1d9 e12fn6ie0')
    if not listing_container:
        print(f"No listing container found on page {i}")
    else:
        print(f"Found {len(listing_container)} listing containers on page {i}")

    # Extract individual listing URLs
    for section in listing_container:
        links = section.find_all('a', class_='css-16vl3c1 e17g0c820')
        for link in links:
            href = link['href']
            if href and href not in houselinks:
                # Ensure the URL is complete
                if not href.startswith('http'):
                    href = 'https://www.otodom.pl' + href
                houselinks.append(href)

print(f"Total houselinks found: {len(houselinks)}")

# Step 2: Scrape details from each listing
houselist = []
for link in houselinks:
    r = requests.get(link, headers=headers)
    if r.status_code == 403:
        print(f"Access denied for link: {link}")
        continue

    soup = BeautifulSoup(r.text, 'lxml')
    body = soup.select_one('main')
    if not body:
        print(f"No main section found for {link}")
        continue

    # Check if no listings found
    h3_element = body.find('h3', class_='css-1nw1os0 e1ysrxc23')
    if h3_element and h3_element.get_text(strip=True) == 'Nie znaleźliśmy żadnych ogłoszeń':
        print("No advertisements found on this detail page. Stopping the scraping process.")
        break

    try:
        NumerId = link

        nazwa = body.find('h1', class_='css-1wnihf5 e1j8g12x8')
        nazwa = nazwa.text.strip() if nazwa else "N/A"

        cenaM2 = body.find('div', class_='css-1h1l5lm e1j8g12x9')
        cenaM2 = cenaM2.text.strip() if cenaM2 else "N/A"

        cena = body.find('strong', class_='css-t3wmkv e9aa0kv0')
        cena = cena.text.strip() if cena else "N/A"

        adres = body.find('a', class_='eozeyij0 css-1helwne e1p0dzoz0')
        adres = adres.text.strip() if adres else "N/A"

        powierzchnia = body.find('div', {'aria-label': 'Powierzchnia', 'class': 'css-1ivc1bc e26jmad1'})
        powierzchnia = powierzchnia.text.strip() if powierzchnia else "N/A"

        formaWlasnosci = body.find('div', {'aria-label': 'Forma własności', 'class': 'css-1ivc1bc e26jmad1'})
        formaWlasnosci = formaWlasnosci.text.strip() if formaWlasnosci else "N/A"

        LiczbaPokoi = body.find('div', {'aria-label': 'Liczba pokoi', 'class': 'css-1ivc1bc e26jmad1'})
        LiczbaPokoi = LiczbaPokoi.text.strip() if LiczbaPokoi else "N/A"

        stanWykonczenia = body.find('div', {'aria-label': 'Stan wykończenia','class':'css-1ivc1bc e26jmad1'})
        stanWykonczenia = stanWykonczenia.text.strip() if stanWykonczenia else "N/A"

        pietro = body.find('div', {'aria-label': 'Piętro', 'class': 'css-1ivc1bc e26jmad1'})
        pietro = pietro.text.strip() if pietro else "N/A"

        BalkonOgrodTaras = body.find('div', {'aria-label': 'Balkon / ogród / taras', 'class': 'css-1ivc1bc e26jmad1'})
        BalkonOgrodTaras = BalkonOgrodTaras.text.strip() if BalkonOgrodTaras else "N/A"

        Czynsz = body.find('div', {'aria-label': 'Czynsz', 'class': 'css-1ivc1bc e26jmad1'})
        Czynsz = Czynsz.text.strip() if Czynsz else "N/A"
        
        MiejsceParkingowe = body.find('div', {'aria-label': 'Miejsce parkingowe', 'class': 'css-1ivc1bc e26jmad1'})
        MiejsceParkingowe = MiejsceParkingowe.text.strip() if MiejsceParkingowe else "N/A"
        
        ObslugaZdalna = body.find('div', {'aria-label': 'Obsługa zdalna', 'class': 'css-1ivc1bc e26jmad1'})
        ObslugaZdalna = ObslugaZdalna.text.strip() if ObslugaZdalna else "N/A"

        ogrzewanie = body.find('div', {'aria-label': 'Ogrzewanie', 'class': 'css-1ivc1bc e26jmad1'})
        ogrzewanie = ogrzewanie.text.strip() if ogrzewanie else "N/A"

        Rynek = body.find('div', {'aria-label': 'Rynek', 'class': 'css-1ivc1bc e26jmad1'})
        Rynek = Rynek.text.strip() if Rynek else "N/A"

        TypOgloszeniodawcy = body.find('div', {'aria-label': 'Typ ogłoszeniodawcy', 'class': 'css-1ivc1bc e26jmad1'})
        TypOgloszeniodawcy = TypOgloszeniodawcy.text.strip() if TypOgloszeniodawcy else "N/A"

        DostepneOd = body.find('div', {'aria-label': 'Dostępne od', 'class': 'css-1ivc1bc e26jmad1'})
        DostepneOd = DostepneOd.text.strip() if DostepneOd else "N/A"

        RokBudowy = body.find('div', {'aria-label': 'Rok budowy', 'class': 'css-1ivc1bc e26jmad1'})
        RokBudowy = RokBudowy.text.strip() if RokBudowy else "N/A"

        RodzajZabudowy = body.find('div', {'aria-label': 'Rodzaj zabudowy', 'class': 'css-1ivc1bc e26jmad1'})
        RodzajZabudowy = RodzajZabudowy.text.strip() if RodzajZabudowy else "N/A"
        
        Okna = body.find('div', {'aria-label': 'Okna', 'class': 'css-1ivc1bc e26jmad1'})
        Okna = Okna.text.strip() if Okna else "N/A"

        CzyWinda = body.find('div', {'aria-label': 'Winda', 'class': 'css-1ivc1bc e26jmad1'})
        CzyWinda = CzyWinda.text.strip() if CzyWinda else "N/A"
        
        Media = body.find('div', {'aria-label': 'Media', 'class': 'css-1ivc1bc e26jmad1'})
        Media = Media.text.strip() if Media else "N/A"

        Zabezpieczenia = body.find('div', {'aria-label': 'Zabezpieczenia', 'class': 'css-1ivc1bc e26jmad1'})
        Zabezpieczenia = Zabezpieczenia.text.strip() if Zabezpieczenia else "N/A"

        Wyposazenie = body.find('div', {'aria-label': 'Wyposażenie', 'class': 'css-1ivc1bc e26jmad1'})
        Wyposazenie = Wyposazenie.text.strip() if Wyposazenie else "N/A"

        InformacjeDodatkowe = body.find('div', {'aria-label': 'Informacje dodatkowe', 'class': 'css-1ivc1bc e26jmad1'})
        InformacjeDodatkowe = InformacjeDodatkowe.text.strip() if InformacjeDodatkowe else "N/A"

        MaterialBudynku = body.find('div', {'aria-label': 'Materiał budynku', 'class': 'css-1ivc1bc e26jmad1'})
        MaterialBudynku = MaterialBudynku.text.strip() if MaterialBudynku else "N/A"

        rooms = {
            'ID': NumerId,
            'Nazwa': nazwa,
            'CenaM2': cenaM2,
            'Cena': cena,
            'Adres': adres,
            'Powierzchnia': powierzchnia,
            'Forma_Wlasnosci': formaWlasnosci,
            'LiczbaPokoi': LiczbaPokoi,
            'StanWykonczenia': stanWykonczenia,
            'Pietro': pietro,
            'BalkonOgrodTaras': BalkonOgrodTaras,
            'Czynsz': Czynsz,
            'Ogrzewanie': ogrzewanie,
            'Rynek': Rynek,
            'TypOgloszeniodawcy': TypOgloszeniodawcy,
            'DostepneOd': DostepneOd,
            'RokBudowy': RokBudowy,
            'RodzajZabudowy': RodzajZabudowy,
            'CzyWinda': CzyWinda,
            'Zabezpieczenia': Zabezpieczenia,
            'Wyposazenie': Wyposazenie,
            'InformacjeDodatkowe': InformacjeDodatkowe,
            'MaterialBudynku': MaterialBudynku
        }
        houselist.append(rooms)
    except AttributeError as e:
        #print(f"Error parsing details for {link}: {e}")
        continue

# Save the data to a DataFrame
pd.set_option('display.max_colwidth', None)
df = pd.DataFrame(houselist)
df

Page 4246 status code: 200
Found 39 listing containers on page 4246
Page 4247 status code: 200
Found 5 listing containers on page 4247
Page 4248 status code: 200
No advertisements found. Stopping the scraping process.
Total houselinks found: 44


,ID,Nazwa,CenaM2,Cena,Adres,Powierzchnia,Forma_Wlasnosci,LiczbaPokoi,StanWykonczenia,Pietro,...,Rynek,TypOgloszeniodawcy,DostepneOd,RokBudowy,RodzajZabudowy,CzyWinda,Zabezpieczenia,Wyposazenie,InformacjeDodatkowe,MaterialBudynku
0,https://www.otodom.pl/pl/oferta/3-pokojowe-mieszkanie-59m2-ogrodek-ID4pMpd,3-pokojowe mieszkanie 59m2 + ogródek,,Zapytaj o cenę,"ul. Falencka, Falenty Nowe, Raszyn, pruszkowski, mazowieckie",Powierzchnia59 m²,Forma własnościpełna własność,Liczba pokoi3,Stan wykończeniado wykończenia,Piętro1/1,...,N/A,N/A,N/A,N/A,N/A,N/A,N/A,N/A,N/A,N/A
1,https://www.otodom.pl/pl/oferta/mieszkanie-72-m2-wioska-gotowe-do-zamieszkania-ID4r5b4,Mieszkanie 72 m2 - WIOSKA - Gotowe do Zamieszkania,7 625 zł/m²,549 000 zł,"Wioska, Syców, oleśnicki, dolnośląskie",Powierzchnia72 m²,Forma własnościpełna własność,Liczba pokoi3,Stan wykończeniado zamieszkania,Piętroparter/2,...,N/A,N/A,N/A,N/A,N/A,N/A,N/A,N/A,N/A,N/A
2,https://www.otodom.pl/pl/oferta/1-pokojowe-mieszkanie-41m2-ogrodek-bez-prowizji-ID4qFvC,1-pokojowe mieszkanie 41m2 + ogródek Bez Prowizji,18 200 zł/m²,749 840 zł,"ul. Mieszczańska, Nadodrze, Śródmieście, Wrocław, dolnośląskie","Powierzchnia41,2 m²",Forma własnościpełna własność,Liczba pokoi1,Stan wykończeniado wykończenia,Piętroparter/6,...,N/A,N/A,N/A,N/A,N/A,N/A,N/A,N/A,N/A,N/A
3,https://www.otodom.pl/pl/oferta/ursynow-kabaty-trzy-pokoje-75-m2-z-garazem-ID2Bask,Ursynów Kabaty trzy pokoje 75 m2 z garażem,16 733 zł/m²,1 260 000 zł,"Pyry, Ursynów, Warszawa, mazowieckie","Powierzchnia75,3 m²",Forma własnościpełna własność,Liczba pokoi3,Stan wykończeniaZapytaj,Piętro4/5,...,N/A,N/A,N/A,N/A,N/A,N/A,N/A,N/A,N/A,N/A
4,https://www.otodom.pl/pl/oferta/mieszkanie-200-m-warszawa-ID2f1sl,"Mieszkanie, 200 m², Warszawa",13 500 zł/m²,2 700 000 zł,"ul. Łotewska, Saska Kępa, Praga-Południe, Warszawa, mazowieckie",Powierzchnia200 m²,Forma własnościpełna własność,Liczba pokoi8,Stan wykończeniado remontu,Piętro1/1,...,N/A,N/A,N/A,N/A,N/A,N/A,N/A,N/A,N/A,N/A
5,https://www.otodom.pl/pl/oferta/czarnieckiego-przestronne-m-4-z-balkonem-winda-ID4r534,"Czarnieckiego przestronne M-4, z balkonem, winda",7 700 zł/m²,438 900 zł,"Stare Polesie, Polesie, Łódź, łódzkie",Powierzchnia57 m²,Forma własnościpełna własność,Liczba pokoi3,Stan wykończeniado zamieszkania,Piętro5/10,...,N/A,N/A,N/A,N/A,N/A,N/A,N/A,N/A,N/A,N/A
6,https://www.otodom.pl/pl/oferta/abrahama-15-wladyslawowo-ID4qj6u,Abrahama 15 Władysławowo,12 344 zł/m²,790 000 zł,"ul. Abrahama, Władysławowo, Władysławowo, pucki, pomorskie",Powierzchnia64 m²,Forma własnościpełna własność,Liczba pokoi3,Stan wykończeniado zamieszkania,Piętroparter/3,...,N/A,N/A,N/A,N/A,N/A,N/A,N/A,N/A,N/A,N/A
7,https://www.otodom.pl/pl/oferta/rabka-zdroj-mieszkanie-48m2-ID4qODF,Rabka-Zdrój mieszkanie 48m2,9 667 zł/m²,464 000 zł,"Rabka-Zdrój, Rabka-Zdrój, nowotarski, małopolskie",Powierzchnia48 m²,Forma własnościspółdzielcze wł. prawo do lokalu,Liczba pokoi3,Stan wykończeniado remontu,Piętro2/4,...,N/A,N/A,N/A,N/A,N/A,N/A,N/A,N/A,N/A,N/A
8,https://www.otodom.pl/pl/oferta/sprzedam-mieszkanie-w-centrum-rzeszowa-cegielnian-ID4qK2j,Sprzedam mieszkanie w CENTRUM RZESZOWA Cegielnian,12 500 zł/m²,650 000 zł,"Nowe Miasto, Rzeszów, podkarpackie",Powierzchnia52 m²,Forma własnościpełna własność,Liczba pokoi2,Stan wykończeniado zamieszkania,Piętro1/3,...,N/A,N/A,N/A,N/A,N/A,N/A,N/A,N/A,N/A,N/A
9,https://www.otodom.pl/pl/oferta/mieszkanie-62mb-super-lokalizacja-klimatyzacja-ID4r2l1,Mieszkanie 62mb super lokalizacja klimatyzacja,5 565 zł/m²,345 000 zł,"Dębica, dębicki, podkarpackie",Powierzchnia62 m²,Forma własnościpełna własność,Liczba pokoi4,Stan wykończeniado zamieszkania,Piętro4/4,...,N/A,N/A,N/A,N/A,N/A,N/A,N/A,N/A,N/A,N/A


In [150]:
df_copy = df.copy()

In [ ]:
## FEATURE ENGINEERING

In [151]:
## ID

df_copy['ID'] = df_copy['ID'].apply(lambda x: re.search(r'(ID\w+)', x).group(0) if re.search(r'(ID\w+)', x) else None)

In [152]:
# Cena
df_copy['Cena'] = df_copy['Cena'].apply(lambda y: str(y).split('zł')[0])

In [153]:
# CenaM2
df_copy['CenaM2'] = df_copy['CenaM2'].apply(lambda y: str(y).split('zł/m²')[0])

In [25]:
## Nazwz czy potrzebna ?

In [154]:
df_copy

,ID,Nazwa,CenaM2,Cena,Adres,Powierzchnia,Forma_Wlasnosci,LiczbaPokoi,StanWykonczenia,Pietro,...,Rynek,TypOgloszeniodawcy,DostepneOd,RokBudowy,RodzajZabudowy,CzyWinda,Zabezpieczenia,Wyposazenie,InformacjeDodatkowe,MaterialBudynku
0,ID4r58O,Mieszkanie „Park Moniuszki” 3 pokoje 59.11 m.kw.,11 656,689 000,"Mysłowice, śląskie",N/A,N/A,N/A,N/A,N/A,...,N/A,N/A,N/A,N/A,N/A,N/A,N/A,N/A,N/A,N/A
1,ID4qwh3,3-pokojowe mieszkanie 67m2 + loggia Bez Prowizji,17 200,1 162 892,"ul. Wybrzeże Puckie, Pelcowizna, Praga-Północ, Warszawa, mazowieckie",N/A,N/A,N/A,N/A,N/A,...,N/A,N/A,N/A,N/A,N/A,N/A,N/A,N/A,N/A,N/A
2,ID4po5g,2-pokojowe mieszkanie 35m2 + loggia Bezpośrednio,17 300,618 821,"ul. Wybrzeże Puckie, Pelcowizna, Praga-Północ, Warszawa, mazowieckie",N/A,N/A,N/A,N/A,N/A,...,N/A,N/A,N/A,N/A,N/A,N/A,N/A,N/A,N/A,N/A
3,ID4aWND,Al. Św. Wojciecha 59 m2. 3 pok. Oddane do użytku!,6 100,362 706,"Mława, mławski, mazowieckie",N/A,N/A,N/A,N/A,N/A,...,N/A,N/A,N/A,N/A,N/A,N/A,N/A,N/A,N/A,N/A
4,ID4aUGB,"Mieszkanie, 90 m², Kutno",3 889,350 000,"Kutno, kutnowski, łódzkie",N/A,N/A,N/A,N/A,N/A,...,N/A,N/A,N/A,N/A,N/A,N/A,N/A,N/A,N/A,N/A
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
254,ID4qxii,Sprzedam mieszkanie w Zawierciu,5 719,290 000,"Centrum, Zawiercie, zawierciański, śląskie",N/A,N/A,N/A,N/A,N/A,...,N/A,N/A,N/A,N/A,N/A,N/A,N/A,N/A,N/A,N/A
255,ID4qw1q,sprzedam mieszkanie,3 865,160 000,"Koszalińska 5/1, Sianów, Sianów, koszaliński, zachodniopomorskie",N/A,N/A,N/A,N/A,N/A,...,N/A,N/A,N/A,N/A,N/A,N/A,N/A,N/A,N/A,N/A
256,ID4qqjJ,Zamienię kawalerkę na działkę budowlaną,5 181,100 000,"ul. ks. Piotra Wawrzyniaka, Inowrocław, inowrocławski, kujawsko-pomorskie",N/A,N/A,N/A,N/A,N/A,...,N/A,N/A,N/A,N/A,N/A,N/A,N/A,N/A,N/A,N/A
257,ID4qkjq,apartament SOPOT dolny 400 m do plaży i mola,21 000,1 050 000,"ul. Fryderyka Chopina, Dolny Sopot - Grunwaldzka, Dolny Sopot, Sopot, pomorskie",N/A,N/A,N/A,N/A,N/A,...,N/A,N/A,N/A,N/A,N/A,N/A,N/A,N/A,N/A,N/A


In [155]:
## Adres
### Wyodrębnienie nazwy wojewodztwa

wojewodztwa = ['dolnośląskie','kujawsko-pomorskie','lubelskie','lubuskie','łódzkie',
               'małopolskie','mazowieckie','opolskie','podkarpackie','podlaskie','pomorskie',
               'śląskie','świętokrzyskie','warmińsko-mazurskie','wielkopolskie','zachodniopomorskie']



In [156]:
df_copy['Wojewodztwo'] = df_copy['Adres'].str.split().apply(lambda x: '_'.join([m for m in x if m in wojewodztwa])).replace('',np.nan)

In [157]:
### Wyodrębnienie nazwy miast
file_name = r'C:\Users\seba_\Desktop\PythonProject\OtoDom\ListOfCity\SIMC_15-06-2024 ({}).csv'
df_city_poland = pd.concat([pd.read_csv(file_name.format(i),sep=';') for i in range(0, 16)])

In [190]:
city_set = {city.strip().lower() for city in df_city_poland['NAZWA']}

def extract_city_values(address):
    parts = [part.strip().lower() for part in address.split(',')]
    city = next((part for part in parts if part in city_set), np.nan)
    return city.capitalize() if city is not np.nan else city

df_copy['Miasto'] = df_copy['Adres'].apply(extract_city_values)


In [191]:
### Wyodrębnienie nazwy powiatow

In [192]:
df_district_poland = pd.read_excel(r'C:\Users\seba_\Desktop\PythonProject\OtoDom\ListofDistrict\ListaPowiatow.xlsx')

In [193]:
district_set = {district.strip().lower() for district in df_district_poland['Powiat']}

def extract_district_values(address):
    parts = [part.strip().lower() for part in address.split(',')]
    district = next((part for part in parts if part in district_set), np.nan)
    return district.capitalize() if district is not np.nan else district

df_copy['Powiat'] = df_copy['Adres'].apply(extract_district_values)

In [194]:
df_copy[['Miasto','Adres','Powiat','Wojewodztwo']]

,Miasto,Adres,Powiat,Wojewodztwo
0,Mysłowice,"Mysłowice, śląskie",Mysłowice,śląskie
1,Warszawa,"ul. Wybrzeże Puckie, Pelcowizna, Praga-Północ, Warszawa, mazowieckie",Warszawa,mazowieckie
2,Warszawa,"ul. Wybrzeże Puckie, Pelcowizna, Praga-Północ, Warszawa, mazowieckie",Warszawa,mazowieckie
3,Mława,"Mława, mławski, mazowieckie",Mławski,mazowieckie
4,Kutno,"Kutno, kutnowski, łódzkie",Kutnowski,łódzkie
...,...,...,...,...
254,Zawiercie,"Centrum, Zawiercie, zawierciański, śląskie",Zawierciański,śląskie
255,NaN,"Koszalińska 5/1, Sianów, Sianów, koszaliński, zachodniopomorskie",Koszaliński,zachodniopomorskie
256,Inowrocław,"ul. ks. Piotra Wawrzyniaka, Inowrocław, inowrocławski, kujawsko-pomorskie",Inowrocławski,kujawsko-pomorskie
257,Sopot,"ul. Fryderyka Chopina, Dolny Sopot - Grunwaldzka, Dolny Sopot, Sopot, pomorskie",Sopot,pomorskie


In [195]:
### Wyodrębnienie nazwy ulicy

In [196]:
def extract_ulica_values(address):
    matches = re.findall(r'\bul\.\s*[^,]*', address)
    return ', '.join(matches) if matches else np.nan

df_copy['Ulica'] = df_copy['Adres'].apply(extract_ulica_values)

In [199]:
df_copy.head()

,ID,Nazwa,CenaM2,Cena,Adres,Powierzchnia,Forma_Wlasnosci,LiczbaPokoi,StanWykonczenia,Pietro,...,RodzajZabudowy,CzyWinda,Zabezpieczenia,Wyposazenie,InformacjeDodatkowe,MaterialBudynku,Wojewodztwo,Miasto,Powiat,Ulica
0,ID4r58O,Mieszkanie „Park Moniuszki” 3 pokoje 59.11 m.kw.,11 656,689 000,"Mysłowice, śląskie",N/A,N/A,N/A,N/A,N/A,...,N/A,N/A,N/A,N/A,N/A,N/A,śląskie,Mysłowice,Mysłowice,NaN
1,ID4qwh3,3-pokojowe mieszkanie 67m2 + loggia Bez Prowizji,17 200,1 162 892,"ul. Wybrzeże Puckie, Pelcowizna, Praga-Północ, Warszawa, mazowieckie",N/A,N/A,N/A,N/A,N/A,...,N/A,N/A,N/A,N/A,N/A,N/A,mazowieckie,Warszawa,Warszawa,ul. Wybrzeże Puckie
2,ID4po5g,2-pokojowe mieszkanie 35m2 + loggia Bezpośrednio,17 300,618 821,"ul. Wybrzeże Puckie, Pelcowizna, Praga-Północ, Warszawa, mazowieckie",N/A,N/A,N/A,N/A,N/A,...,N/A,N/A,N/A,N/A,N/A,N/A,mazowieckie,Warszawa,Warszawa,ul. Wybrzeże Puckie
3,ID4aWND,Al. Św. Wojciecha 59 m2. 3 pok. Oddane do użytku!,6 100,362 706,"Mława, mławski, mazowieckie",N/A,N/A,N/A,N/A,N/A,...,N/A,N/A,N/A,N/A,N/A,N/A,mazowieckie,Mława,Mławski,NaN
4,ID4aUGB,"Mieszkanie, 90 m², Kutno",3 889,350 000,"Kutno, kutnowski, łódzkie",N/A,N/A,N/A,N/A,N/A,...,N/A,N/A,N/A,N/A,N/A,N/A,łódzkie,Kutno,Kutnowski,NaN
